In [ ]:
import pandas as pd
import numpy as np
from sklearn.decomposition import PCA
from datetime import datetime


## Table 3 
hier komt tabel 3, maar ik was nog even bezig met het bekijken hoe zij deze data hebben verkregen

In [ ]:
# loading dataframes 

monthly_df = pd.read_csv('managed_portfolios_anom_50.csv')
daily_df = pd.read_csv('managed_portfolios_anom_d_50.csv')

# drop all momentum factors or factors that are constructed based on momentum

momentum_list = ['r_mom', 'r_indmom', 'r_valmom', 'r_valmomprof', 'r_mom12', 'r_momrev', 'r_indmomrev']
monthly_df.drop(columns=momentum_list, inplace=True)
daily_df.drop(columns=momentum_list, inplace=True)

# missing list = ['dur', 'divg', 'invaci', 'ipo'] -> deze worden gemaakt uit daily stock data doormiddel van een signals list

# create datetime column and set to YYYY-MM format of full monthly sample

monthly_df['date'] = pd.to_datetime(monthly_df['date'])
monthly_full = monthly_df.set_index('date').to_period('M').reset_index()

# create datetime column and set to YYYY-MM format for monthly sample from July 1973

monthly_1973 = monthly_df[monthly_df['date'] >= pd.Timestamp(1973, 7, 1)]
monthly_1973 = monthly_df.set_index('date').to_period('M').reset_index()

# create datetime column for daily sample 

daily_df['date'] = pd.to_datetime(daily_df['date'])
daily_1973 = daily_df.copy()
daily_full = daily_df.copy()

# create the daily sample from July 1973

daily_1973 = daily_1973[daily_1973['date'] >= pd.Timestamp(1973, 7, 1)]
daily_1973.reset_index(drop=True, inplace=True)


# if there are missing values, we replace them with a 0 (as the paper also mentions) (zie voetnoot 16 KNS)

monthly_full.fillna(0, inplace=True)
daily_full.fillna(0, inplace=True)
monthly_1973.fillna(0, inplace=True)
daily_1973.fillna(0, inplace=True)

print(daily_full.head(5))
print(monthly_full.head(5))

In [ ]:
# deze loop heb ik nagemaakt uit het onderzoek. Volgens mij repliceert dit exact de wijze waarop zij dat ook in het onderzoek in stap 1 en 2 doen. 
# Echter zijn er dus te weinig dimensies aanwezig. Ik zie in hun stappen niet goed hoe zij de data gebruik uit 1963 om de PCs te berekenen. 
# Geen idee hoe dit anders moet. Ik heb wel getest indien er genoeg dimensies zijn, dat werkt gewoon. 

for year in range(1974, 1975):
    for month in range(1, 13):

        # Pakt hier de subsample maand met de daily returns (met gegevens uit de loop)

        subsample = daily_df[(daily_df['date'].dt.year == year) & (daily_df['date'].dt.month == month)]

        # maakt een aparte df aan voor de returns met corresponderende datum (de market return rme en equally weighted return re_ew worden gedropt dus)

        subsample_returns = subsample.drop(['rme', 're_ew'], axis=1)
        subsample_returns.set_index('date', inplace=True)

        # een toevoeging dat subsample niet leeg mag zijn, want onze data begint in July en anders werkt het begin van de loop niet. 

        if not subsample.empty:
            # Deze line extract de factors uit de dataframe. Deze starten allemaal met r_
            factor_vars = [col for col in subsample.columns if col.startswith('r_')]

            # Deze converteert de data zodat alleen de factor returns aanwezig zijn. (Dus haalt effectief de date, rme, en re_ew weg)
            pca_data = subsample[factor_vars]

            # Voert de PCA uit met het aantal components gelijk aan het aantal factoren
            pca_results = PCA(n_components=len(factor_vars)).fit(pca_data)

            # De eigen vectors zijn de components van de PCA, deze komen eruit als een array (dus een lijst in de vorm van een vector: [X1, X2, ..., Xn])
            eigenvectors = pca_results.components_

            # Dit berekent de dagelijkse vector returns, door de daily returns te vermenigvuldigen et de eigenvectors (conform stap 2). 
            pc_factor_returns = np.dot(subsample_returns.values, eigenvectors.T)

            # Deze worden vervolgens in een dataframe gezet 
            pc_factor_returns_df = pd.DataFrame(pc_factor_returns, index=subsample_returns.index)

            # Vervolgens worden ze geresampled (oftewel opgeteld conform de formule gegeven in stap 2). 
            monthly_pc_factor_returns = pc_factor_returns_df.resample('M').sum()
            print(monthly_pc_factor_returns)